In [1]:
import numpy as np
import re
import pandas as pd
from IPython.display import display
import scipy.stats as stats
import os
import time


# function to split files
def fileSplit(dataFile):
    
    #line to split files on
    fileEnd='END OF VARIABLES SECTION'
    
    #counter to delete last empty file
    fileCount = 1
    
    # create files
    def files():
        n = 0
        while True:
            n += 1
            fileCount = n
            yield open('tabular_data/%d.csv' % n, 'w')
    
    #creates and opens first file
    fileSystem=files()
    outFile=next(fileSystem)
    
    #loops through lines in file. If not end of df, writes line to file
    with open(dataFile) as inFile:
        for line in inFile:
            if fileEnd not in line:
                outFile.write(line)
            #if end of dataframe, write line to file and open now file    
            else:
                outFile.write(line)
                outFile=next(fileSystem)
                fileCount += 1
    
    #close and delete last empty file
    outFile.close()
    os.remove('tabular_data/%d.csv' % fileCount)
    
#fileSplit('data/waterTemp2008.csv')

In [2]:
# Function to process(wording, ugh) processed files after original file has been split into processed files
def processSplit(fileName):
    #latitude and longitude variables
    latitude = ''
    longitude = ''
    lineCounter = 0
    df = []
    latColumn = []
    longColumn = []
    
    
    #opens file and reads lines
    file = open(fileName, 'r')
    for line in file:
        if 'Latitude' in line: 
            latitude = line
            lineCounter += 1
        elif 'Longitude' in line: 
            longitude = line
            lineCounter += 1
        elif 'UNITS' in line:
            df = pd.read_csv(fileName, header = lineCounter)
            break
        else:
            lineCounter += 1

    # split line on commas
    latitude = latitude.split(',')
    longitude = longitude.split(',')

    #select item from list with numerical lat/long value, strip whitespace
    latitude = latitude[2]
    longitude = longitude[2]
    latitude = float(latitude.strip())
    longitude = float(longitude.strip())
    
    
    #rename columns
    df = (df.rename(columns = {'m         ': 'Depth(m)', 'degrees C ': 'Temperature(C)', ' .3': 'Salinity(unitless)',
                              ' .4': 'Oxygen(µmol/kg)'}))
    #trim first and last useless rows from df, selects desired columns
    df = df.reindex(columns = ['Depth(m)', 'Temperature(C)', 'Salinity(unitless)', 'Oxygen(µmol/kg)'])
    df = df.iloc[1:(len(df)-2), :]
    
    #loops to create latitude/longitude columns
    for i in range(len(df)):
        latColumn.append(latitude)
        longColumn.append(longitude)
    
    df['Latitude'] = latColumn
    df['Longitude'] = longColumn
    file.close()
    return(df)
    

#processSplit('tabular_data/1.csv')

In [3]:
# function that loops through processed files in tabular_data folder and adds each to dataframe
def processFile(fileList):
    df = []
    # loops through files
    for file in fileList:
        #ignores .gitkeep file to include folder in gitHub
        if file == '.gitkeep': continue
        #creates temporary datafram from each processed file
        addData = processSplit('tabular_data/%s' % file)
        #checks to see if anything is appended to df. If not, creates df with same dimensions as processed df's, then adds data
        if len(df) == 0:
            df = pd.DataFrame().reindex(columns = addData.columns)
        df = df.append(addData, ignore_index = True)
    
    # cleans processed files out of tabular_data for next data file to run
    for file in fileList:
        if file == '.gitkeep': continue
        os.remove('tabular_data/%s' %file)
            
            
    return df
    
    
    
#dirFiles = os.listdir('tabular_data/')
#display(processFile(dirFiles))

In [8]:
# pulls list of files in data/ folder
uncutFiles = os.listdir('data/')

# declare empty variable for dataframe, switch to append to dataframe after it is created
df=[]
newData=True
dfCount = 0

# loop through files in data/ folder
for file in uncutFiles:
    # create and log temporary files
    fileSplit('data/%s' %file)
    cutFiles = os.listdir('tabular_data/')
    # get month and year to add as df column
    monthYear = file.split('_')
    monthYear[1] = monthYear[1][:4]
    #process temporary files into dataframe
    addData = processFile(cutFiles)
    # empty lists for month and year columns to append to df
    addMonth = []
    addYear = []
    #fill lists with month and year equal to length of df to append
    for i in range(len(addData)):
        addMonth.append(monthYear[0])
        addYear.append(monthYear[1])
    #append columns to df
    addData['Month'] = addMonth
    addData['Year'] = addYear
    dfCount += len(addData)
    #checks to either create df or append data to df
    if newData:
        df = addData
        newData = False
        print('New df from {} with {} lines'.format(file, len(addData)))
    else: 
        df = df.append(addData, ignore_index = True)
        print('Added to df from {} with {} lines'.format(file, len(addData)))

print(dfCount)
display(df)

New df with January_1987.csv with 34846 lines
Added to df with January_2007.csv with 900 lines
Added to df with January_2017.csv with 7279 lines
Added to df with January_2018.csv with 304 lines
Added to df with July_1987.csv with 79377 lines
Added to df with July_1997.csv with 23172 lines
Added to df with July_2017.csv with 4974 lines
Added to df with July_2018.csv with 477 lines
151329


,Depth(m),Temperature(C),Salinity(unitless),Oxygen(µmol/kg),Latitude,Longitude,Month,Year
0,0.,2.83,34.320,NaN,68.8500,37.3333,January,1987
1,5.,2.84,34.319,NaN,68.8500,37.3333,January,1987
2,10.,2.84,34.318,NaN,68.8500,37.3333,January,1987
3,15.,2.84,34.317,NaN,68.8500,37.3333,January,1987
4,20.,2.84,34.316,NaN,68.8500,37.3333,January,1987
...,...,...,...,...,...,...,...,...
151324,25.,3.0509,6.90530,331.,59.5298,24.9702,July,2018
151325,30.,2.7945,7.29035,333.,59.5298,24.9702,July,2018
151326,0.0,14.6,7.6,318.,54.2040,15.5570,July,2018
151327,0.0,12.5,7.6,301.,54.2950,16.1250,July,2018


In [4]:
fileName = 'data/July_1987.csv'
fileSplit(fileName)
splitList=os.listdir('tabular_data')
display(processFile(splitList))


,Depth(m),Temperature(C),Salinity(unitless),Oxygen(µmol/kg),Latitude,Longitude
0,0.,21.0,32.47,NaN,34.565,134.8967
1,5.,20.9,32.47,NaN,34.565,134.8967
2,10.,20.7,32.48,NaN,34.565,134.8967
3,15.,20.7,32.49,NaN,34.565,134.8967
4,20.,20.6,32.52,NaN,34.565,134.8967
...,...,...,...,...,...,...
79372,75.0,5.8500,18.5000,NaN,44.617,33.1000
79373,80.0,5.9900,18.6200,NaN,44.617,33.1000
79374,85.0,6.3400,18.8200,NaN,44.617,33.1000
79375,90.0,6.6600,19.0000,NaN,44.617,33.1000
